In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import gensim
import gensim.downloader as api
import nltk
from nltk.tokenize import word_tokenize
import spacy
from hashlib import md5

In [2]:
nlp = spacy.load('fr_core_news_lg')

In [3]:
data = pd.read_csv("final_jobs_data.csv")
data_org = pd.read_csv("job_data.csv")

In [4]:
text_elements = data_org['Title'].tolist()
similarity_matrix = pd.DataFrame(index=text_elements, columns=text_elements)

In [5]:
print(data.shape)
data.head()

(1231, 11)


,Title,Description,Experience,School level,Contract Type,Teleworking,Deadline,Entreprise,Post information,Profile information,Personality
0,développeur full stack php casablanca maroc,informatique electroniqu télécom réseau secteu...,1 an 1 3 an,bac plus 2 minimum master ecole ingénieur,cdi,hybride,18 09 2023,entreprise h f associate 10 important cabinet ...,poste participer développement maintenance app...,profil rechercher expérience avérer développem...,recherche nouveauté ambition besoin réflexion ...
1,area manager casablanca maroc,communication publicité rp marketing secteur d...,3 5 an,bac plus 5 minimum master,cdi,non,18 09 2023,entreprise 20 an orchestrer spécialiste mode e...,poste chercher compte un area manager mission ...,profil rechercher bacplus 3 plus 5 marketing m...,organisation flexibilité ambition extraversion...
2,stagiaire marketing casablanca maroc,marketing secteur communication evénementiel,1 an,bac plus 3 minimum université,cdi,non,18 09 2023,entreprise 20 an orchestrer spécialiste mode e...,poste recherchon un stagiaire marketing rejoin...,profil rechercher prérequis formation digital ...,organisation flexibilité ambition extraversion...
3,support it sénior plus 5 an expérience rabat m...,informatique electroniqu secteur informatique,5 10 an,bac plus 2 minimum formation professionnel for...,cdi,non,18 09 2023,entreprise xpertize bureau recrutement unique ...,poste administrer maintenir réseau client pare...,profil rechercher diplôme niveau bacplus 3 adm...,recherche nouveauté ambition besoin réflexion ...
4,agent logistique casablanca maroc,logistique transport métier secteur transport ...,1 3 an,bac plus 2 minimum formation professionnel for...,cdi,non,18 09 2023,entreprise 20 an orchestrer spécialiste mode e...,poste chercher compte agent logistique expérie...,profil rechercher expérience requérir 2 an min...,respect règle besoin action implication travai...


In [6]:
text_columns = ["Title", "Description", "Experience", "School level", "Contract Type", "Teleworking", "Deadline", "Entreprise", "Post information", "Profile information", "Personality"]
data[text_columns] = data[text_columns].fillna('')  # Fill NaN values with an empty string

data['text'] = data[text_columns].apply(lambda row: ' '.join(row), axis=1)

vectorizer = CountVectorizer()
bag_of_words = vectorizer.fit_transform(data['text'])

bag_of_words_df = pd.DataFrame(bag_of_words.toarray(), columns=vectorizer.get_feature_names_out())

bag_of_words_df.to_csv('bag_of_words.csv', index=False)


In [7]:
bag = pd.read_csv("bag_of_words.csv")

In [8]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(data)
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)
df     

C:\Users\jihad\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,contract,deadline,description,entreprise,experience,information,level,personality,post,profile,school,teleworking,text,title,type
0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,1.0,0.000000
1,0.000000,0.0,1.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
2,0.000000,0.0,0.0,0.0,1.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
3,0.000000,0.0,0.0,0.0,0.0,0.000000,0.707107,0.0,0.000000,0.000000,0.707107,0.0,0.0,0.0,0.000000
4,0.707107,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.707107
5,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.000000
6,0.000000,1.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
7,0.000000,0.0,0.0,1.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
8,0.000000,0.0,0.0,0.0,0.0,0.651521,0.000000,0.0,0.758631,0.000000,0.000000,0.0,0.0,0.0,0.000000
9,0.000000,0.0,0.0,0.0,0.0,0.651521,0.000000,0.0,0.000000,0.758631,0.000000,0.0,0.0,0.0,0.000000


In [9]:
vectorizer = TfidfVectorizer()


for j in range(0,11):
    vectors = vectorizer.fit_transform(data.iloc[:,j])
    feature_names = vectorizer.get_feature_names()
    dense = vectors.todense()
    denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)
df


C:\Users\jihad\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,action,ambition,autonomie,besoin,conventionnel,créativité,distance,equilibre,extraversion,flexibilité,...,respect,règle,réflexion,sensibilité,spontanéité,travail,ténacité,volonté,émotionnel,équipe
0,0.00000,0.336802,0.372400,0.514615,0.0,0.0,0.00000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.34045,0.0,0.0,0.310710,0.0,0.0,0.000000,0.0
1,0.00000,0.400357,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.429454,0.505966,...,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
2,0.00000,0.400357,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.429454,0.505966,...,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
3,0.00000,0.336802,0.372400,0.514615,0.0,0.0,0.00000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.34045,0.0,0.0,0.310710,0.0,0.0,0.000000,0.0
4,0.59126,0.000000,0.000000,0.249007,0.0,0.0,0.00000,0.0,0.000000,0.000000,...,0.377572,0.377572,0.00000,0.0,0.0,0.300687,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,0.00000,0.336802,0.372400,0.514615,0.0,0.0,0.00000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.34045,0.0,0.0,0.310710,0.0,0.0,0.000000,0.0
1227,0.00000,0.336802,0.372400,0.514615,0.0,0.0,0.00000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.34045,0.0,0.0,0.310710,0.0,0.0,0.000000,0.0
1228,0.00000,0.336802,0.372400,0.514615,0.0,0.0,0.00000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.34045,0.0,0.0,0.310710,0.0,0.0,0.000000,0.0
1229,0.00000,0.000000,0.338796,0.468179,0.0,0.0,0.00000,0.0,0.328679,0.000000,...,0.000000,0.000000,0.00000,0.0,0.0,0.282673,0.0,0.0,0.000000,0.0


In [10]:
mising_values = data.isnull().sum()
mising_values

Title                  0
Description            0
Experience             0
School level           0
Contract Type          0
Teleworking            0
Deadline               0
Entreprise             0
Post information       0
Profile information    0
Personality            0
text                   0
dtype: int64

In [11]:
vectorizer = TfidfVectorizer()

for j in range(0,11):
    vectors = vectorizer.fit_transform(data.iloc[:,j])
    print(vectorizer.vocabulary_)


{'développeur': 396, 'full': 520, 'stack': 1082, 'php': 866, 'casablanca': 217, 'maroc': 709, 'area': 100, 'manager': 703, 'stagiaire': 1084, 'marketing': 708, 'support': 1104, 'it': 641, 'sénior': 1118, 'plus': 885, 'an': 69, 'expérience': 469, 'rabat': 938, 'agent': 46, 'logistique': 688, 'devenir': 353, 'consulter': 296, 'rejoindre': 957, 'leyton': 676, 'offrir': 805, 'ouvert': 828, 'ingénieur': 608, 'docteur': 374, 'doctorant': 375, 'informatique': 604, 'expert': 463, 'finance': 485, 'digital': 361, 'fintech': 488, 'responsable': 973, 'magasin': 693, 'kenitra': 658, 'comptable': 282, 'heure': 562, 'fullstack': 521, 'laravel': 666, 'freelance': 511, 'cdi': 227, 'senior': 1044, 'busines': 190, 'intelligence': 621, 'pôle': 929, 'conseiller': 294, 'clientèle': 260, 'francophone': 509, 'developer': 349, 'odoo': 800, 'software': 1064, 'engineer': 432, 'méthode': 771, 'usinage': 1200, 'programmation': 911, 'change': 239, 'management': 702, 'officer': 804, 'testeur': 1150, 'automaticien': 

{'entreprise': 1336, 'associate': 436, '10': 6, 'important': 1795, 'cabinet': 642, 'recrutement': 2854, 'maroc': 2154, 'recruter': 2855, 'partenaire': 2514, 'an': 346, 'accompagner': 193, 'recherche': 2846, 'meilleur': 2191, 'profil': 2727, 'jusqu': 1980, 'intégration': 1913, 'candidat': 661, 'choisir': 756, 'généraliste': 1677, 'intervenir': 1905, 'secteur': 3088, 'activité': 235, 'type': 3452, 'poste': 2653, 'auprès': 480, 'taille': 3302, 'grand': 1649, 'groupe': 1660, 'international': 1898, 'eti': 1370, 'pme': 2618, 'local': 2068, 'qualité': 2818, 'exprimer': 1432, 'travers': 3436, 'méthodologie': 2328, 'résultat': 3028, 'taux': 3314, 'confirmation': 905, '95': 176, 'pourcent': 2660, 'acquisition': 221, 'talent': 3304, 'jamais': 1955, 'bien': 568, 'porter': 2639, 'nom': 2364, 'info': 1836, 'ici': 1765, 'www': 3613, 'hf': 1713, 'associates': 437, 'com': 824, '20': 63, 'orchestrer': 2460, 'spécialiste': 3208, 'mode': 2239, 'enfant': 1300, 'puériculture': 2804, 'franc': 1577, 'distingu

{'poste': 6633, 'participer': 6271, 'développement': 3024, 'maintenance': 5283, 'application': 676, 'web': 9502, 'baser': 1073, 'php': 6447, 'concevoir': 1960, 'coder': 1738, 'tester': 8845, 'fonctionnalité': 3799, 'nouveau': 5908, 'existant': 3484, 'collaborer': 1767, 'équipe': 9725, 'multidisciplinaire': 5727, 'résoudre': 7840, 'problème': 6788, 'technique': 8782, 'assurer': 861, 'qualité': 7116, 'code': 1737, 'effectuer': 3100, 'test': 8842, 'régulier': 7804, 'garantir': 3967, 'performance': 6366, 'optimal': 6071, 'meilleur': 5469, 'pratique': 6669, 'norme': 5885, 'industrie': 4570, 'chercher': 1601, 'compte': 1931, 'un': 9166, 'area': 762, 'manager': 5312, 'mission': 5572, 'vendre': 9333, 'promouvoir': 6891, 'article': 790, 'magasin': 5264, 'accueillir': 192, 'conseiller': 2048, 'client': 1687, 'contrôler': 2189, 'mettre': 5511, 'rayon': 7217, 'manquant': 5333, 'réassort': 7755, 'exposer': 3534, 'marchandise': 5352, 'merchandising': 5488, 'développer': 3027, 'chiffre': 1611, 'affai

{'recherche': 21, 'nouveauté': 15, 'ambition': 1, 'besoin': 3, 'réflexion': 24, 'autonomie': 2, 'implication': 11, 'travail': 27, 'organisation': 17, 'flexibilité': 9, 'extraversion': 8, 'respect': 22, 'règle': 23, 'action': 0, 'volonté': 29, 'persuasion': 19, 'équipe': 31, 'improvisation': 12, 'créativité': 5, 'objectivité': 16, 'conventionnel': 4, 'distance': 6, 'émotionnel': 30, 'rationalisme': 20, 'intuition': 14, 'spontanéité': 26, 'ténacité': 28, 'sensibilité': 25, 'humilité': 10, 'introversion': 13, 'equilibre': 7, 'personnel': 18}


In [12]:
tfidf_features = {}  # Dictionary to store features and IDF values for each column

In [13]:
for j in range(12):
    vectors = vectorizer.fit_transform(data.iloc[:, j])
    feature_names = vectorizer.get_feature_names_out()
    feature_names_idx = vectorizer.get_feature_names()

    # Dictionary to store features and their IDF values for the current column
    column_features = {}

    for word, word_idx in zip(feature_names, feature_names_idx):
        word_indx = vectorizer.vocabulary_.get(word)
        word_idf = vectorizer.idf_[word_indx]
        column_features[word_idx] = word_idf

    # Save the features and IDF values for the current column in the main dictionary
    tfidf_features[f"Column_{j}"] = column_features

C:\Users\jihad\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [14]:
print(tfidf_features["Column_11"] )

{'00': 3.772588722239781, '000': 2.415950570702779, '00h00': 7.423246963533519, '01': 4.650658241293739, '01h00': 7.423246963533519, '02': 5.408343942991255, '03': 4.091042453358316, '04': 4.332204510175204, '05': 3.2801122371419873, '06': 3.911701524702499, '0602396685': 7.423246963533519, '0608842316': 7.423246963533519, '0620': 7.423246963533519, '0665450938': 7.017781855425356, '07': 2.4923766379061263, '08': 1.706219262127298, '09': 1.921988752988793, '0d': 7.423246963533519, '10': 2.0502860539897174, '100': 2.6783148351702697, '1000': 5.171955164927025, '10000': 7.423246963533519, '100c': 7.423246963533519, '102': 6.506956231659364, '106': 7.423246963533519, '11': 3.5012736272522056, '1100': 4.9383403137455195, '111': 4.820557278089137, '11135': 7.423246963533519, '12': 3.203739258357413, '120': 6.730099782973574, '1200': 5.120661870539474, '1250': 5.408343942991255, '12c': 7.017781855425356, '12h': 7.423246963533519, '12h00': 7.423246963533519, '13': 3.8968864389173583, '1300': 

In [15]:
print(vectors)

  (0, 11963)	0.049961920199354096
  (0, 5963)	0.06248066694791524
  (0, 1371)	0.06750661804058365
  (0, 10326)	0.06487720746163914
  (0, 1604)	0.08756782197896185
  (0, 823)	0.06265964134396075
  (0, 7957)	0.054574051014704826
  (0, 1983)	0.058486271809214604
  (0, 9702)	0.06979892447916562
  (0, 9640)	0.11245765644462045
  (0, 3161)	0.05477150769109771
  (0, 8778)	0.07259204191470021
  (0, 4411)	0.05240283362959221
  (0, 8452)	0.061703601230084754
  (0, 4823)	0.03827981428751392
  (0, 11035)	0.053847605163665976
  (0, 3948)	0.059139960490809745
  (0, 9864)	0.04463473965126507
  (0, 709)	0.07205205581946987
  (0, 8476)	0.06887882556908208
  (0, 12963)	0.06165663644500022
  (0, 9323)	0.09447992115394671
  (0, 9205)	0.03328253482753156
  (0, 4647)	0.09735147329861447
  (0, 8879)	0.1020330127269497
  :	:
  (1230, 4042)	0.06022339339451146
  (1230, 1624)	0.03259313745185362
  (1230, 6550)	0.06870788807608434
  (1230, 8927)	0.036852561653523054
  (1230, 10404)	0.04117081300384593
  (1230, 9

In [16]:
cos_sim = cosine_similarity(vectors)

In [17]:
print(f'the cosine similarity between job applicaions is {cos_sim[1230,6]}')

the cosine similarity between job applicaions is 0.1026876585844179


In [18]:
job1_similar_list=[]
job2_similar_list=[]
value_similar_list=[]
for i in range(1231):
    for j in range(1231):
        similarity_matrix.loc[data_org['Title'][i], data_org['Title'][j]] = cos_sim[i,j]
        job1_similar_list.append(data['Title'][i])
        job2_similar_list.append(data['Title'][j])
        value_similar_list.append(cos_sim[i,j])
   

In [19]:
job1_similar_df = pd.DataFrame(job1_similar_list, columns=['Job offer 1'])
job2_similar_df = pd.DataFrame(job2_similar_list, columns=['Job offer 2'])
value_similar_df =pd.DataFrame(value_similar_list, columns=['Similarity value'])

In [20]:
cosine_Sim_df=pd.concat([job1_similar_df,job2_similar_df,value_similar_df],axis=1)
cosine_Sim_df['Similarity value']

filtered_df = cosine_Sim_df[(cosine_Sim_df['Similarity value'] > 0.9)]
filtered_df

,Job offer 1,Job offer 2,Similarity value
0,développeur full stack php casablanca maroc,développeur full stack php casablanca maroc,1.0
1232,area manager casablanca maroc,area manager casablanca maroc,1.0
2464,stagiaire marketing casablanca maroc,stagiaire marketing casablanca maroc,1.0
3696,support it sénior plus 5 an expérience rabat m...,support it sénior plus 5 an expérience rabat m...,1.0
4928,agent logistique casablanca maroc,agent logistique casablanca maroc,1.0
...,...,...,...
1510432,ingénieur étude développement java sénior heur...,ingénieur étude développement java sénior heur...,1.0
1511664,ingénieur production casablanca maroc,ingénieur production casablanca maroc,1.0
1512896,ingénieur virtualisation casablanca maroc,ingénieur virtualisation casablanca maroc,1.0
1514128,stagiaire recrutement rabat maroc,stagiaire recrutement rabat maroc,1.0


In [21]:
similarity_matrix = pd.DataFrame(similarity_matrix.fillna(0))
similarity_matrix

,Développeur Full Stack PHP | Casablanca (Maroc),AREA MANAGER | CASABLANCA (Maroc),STAGIAIRE EN MARKETING | CASABLANCA (Maroc),Support IT sénior ( +5 ans d'expérience) | Rabat (Maroc),AGENT LOGISTIQUE | CASABLANCA (Maroc),"Devenez Consultant en rejoignant Leyton ! (Offre ouvertes aux ingénieurs, docteurs et doctorants en informatique) | Casablanca (Maroc)",Expert.e en Finance Digital (FinTech) | Rabat (Maroc),Responsable Magasin | Kenitra (Maroc),Comptable H/F | Casablanca (Maroc),Développeur Fullstack PHP Laravel (Freelance ou CDI) | Casablanca (Maroc),...,Administrateur Système SharePoint et O365 avancés | Technopolis (Maroc),Chargé(e) de Communication | Salé (Technopolis) (Maroc),Full stack PHP/Javascript senior 5 ans d'expérience ( full remote) | Casablanca (Maroc),Ingénieur études et développement JAVA Confirmé (H/F) | Tanger/ Tétouan (Maroc),Développeur PHP | Rabat (Maroc),Ingénieur études et développement JAVA Sénior (H/F) | Tanger/ Tétouan (Maroc),Ingénieur de Production | Casablanca (Maroc),Ingénieur Virtualisation | Casablanca (Maroc),Stagiaire Recrutement | Rabat (Maroc),Directeur Qualité Formation Groupe ( Call center) | Casablanca (Maroc)
Développeur Full Stack PHP | Casablanca (Maroc),1.000000,0.061762,0.076642,0.150434,0.047593,0.123066,0.082364,0.053193,0.072166,0.285825,...,0.049016,0.049537,0.322783,0.142501,0.264377,0.145694,0.113215,0.143904,0.189329,0.123561
AREA MANAGER | CASABLANCA (Maroc),0.061762,1.000000,0.635132,0.054818,0.630765,0.064328,0.056500,0.700773,0.059204,0.054256,...,0.020405,0.092741,0.065006,0.076681,0.056374,0.076037,0.077731,0.072837,0.064429,0.111752
STAGIAIRE EN MARKETING | CASABLANCA (Maroc),0.076642,0.635132,1.000000,0.093949,0.543748,0.082253,0.064234,0.539316,0.059298,0.064866,...,0.018742,0.088402,0.076365,0.069713,0.057175,0.069214,0.055918,0.082035,0.136589,0.081152
Support IT sénior ( +5 ans d'expérience) | Rabat (Maroc),0.150434,0.054818,0.093949,1.000000,0.049249,0.091976,0.079308,0.043320,0.051865,0.135138,...,0.048176,0.051042,0.518190,0.105221,0.086492,0.101162,0.125779,0.140086,0.186186,0.084864
AGENT LOGISTIQUE | CASABLANCA (Maroc),0.047593,0.630765,0.543748,0.049249,1.000000,0.051928,0.046696,0.624715,0.062867,0.041957,...,0.020201,0.039955,0.038703,0.062306,0.046294,0.063166,0.058350,0.062554,0.050092,0.076743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ingénieur études et développement JAVA Sénior (H/F) | Tanger/ Tétouan (Maroc),0.145694,0.076037,0.069214,0.101162,0.063166,0.153286,0.081011,0.079892,0.088818,0.213816,...,0.077377,0.122346,0.148874,0.939305,0.149373,1.000000,0.130696,0.140723,0.120465,0.077272
Ingénieur de Production | Casablanca (Maroc),0.113215,0.077731,0.055918,0.125779,0.058350,0.125781,0.105362,0.055455,0.088712,0.156309,...,0.067858,0.096951,0.107273,0.143890,0.106235,0.130696,1.000000,0.420773,0.123847,0.152608
Ingénieur Virtualisation | Casablanca (Maroc),0.143904,0.072837,0.082035,0.140086,0.062554,0.149650,0.091983,0.063555,0.078771,0.158058,...,0.059881,0.077044,0.111276,0.148390,0.109223,0.140723,0.420773,1.000000,0.171369,0.121928
Stagiaire Recrutement | Rabat (Maroc),0.189329,0.064429,0.136589,0.186186,0.050092,0.151678,0.100879,0.052701,0.097048,0.135753,...,0.053847,0.102397,0.162095,0.124993,0.104031,0.120465,0.123847,0.171369,1.000000,0.100525


In [32]:
similarity_matrix.to_csv("Cosine_similarity_matrix.csv", mode = 'w', index=False)


In [22]:
cosine_Sim_df.to_csv("Cosine_similarity.csv", mode = 'w', index=False)